# Introduction to LangChain

In [44]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Initial setup

### Install dependencies

### Set API key for Groq
Click [here](https://console.groq.com/keys) to create API key for Groq, if not already created.

In [45]:
import os, json, re, getpass
from dotenv import load_dotenv

load_dotenv( override=True)

True

In [46]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key: ")

In [47]:
# if "TEST_API_KEY" not in os.environ:
#     os.environ["TEST_API_KEY"] = getpass.getpass("TEST API Key: ")

In [48]:
if os.environ["GROQ_API_KEY"]:
    print(f"Groq API Key exists and begins {os.environ["GROQ_API_KEY"][:4]}")
else:
    print("Groq API Key not set (and this is optional)")

Groq API Key exists and begins gsk_


## LangChain Components

### LLM / ChatModel

In [49]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕███

In [50]:
#Using LangChain
from langchain.chat_models import init_chat_model

model_name = "llama-3.1-8b-instant"
llm = init_chat_model(model_name, model_provider="groq") #Other Llama alternatives available are llama3-8b-8192, llama-3.3-70b-versatile

In [51]:
llm_response = llm.invoke("what is ECG?")
llm_response

AIMessage(content="ECG stands for Electrocardiogram. It is a medical test that measures the electrical activity of the heart over a period of time. The test records the heart's electrical signals as they travel through the heart muscle, allowing doctors to diagnose and monitor various heart conditions.\n\nDuring an ECG, a series of electrodes are placed on the chest, arms, and legs to record the heart's electrical activity. The electrodes detect the tiny electrical signals generated by the heart's contractions and transmit them to a machine that records the signals on a graph. The graph shows the heart's rhythm, rate, and any abnormalities in the electrical activity.\n\nAn ECG can detect a range of heart conditions, including:\n\n1. Arrhythmias (abnormal heart rhythms)\n2. Heart attacks (myocardial infarctions)\n3. Heart failure\n4. Cardiac arrhythmias\n5. Electrolyte imbalances\n6. Heart valve problems\n7. Cardiac hypertrophy (enlargement of the heart muscle)\n\nThere are different ty

In [52]:
print("type of response", type(llm_response))

type of response <class 'langchain_core.messages.ai.AIMessage'>


In [53]:
# print(llm_response.content)
# display(llm_response.response_metadata)
display(llm_response.usage_metadata)

{'input_tokens': 40, 'output_tokens': 331, 'total_tokens': 371}

In [54]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Detect and correct all spelling, grammatical, and punctuation errors in the provided text.\
                  Ensure proper sentence structure, clarity, and readability.\
                  Retain the core message of the original text while making the necessary corrections"),
    HumanMessage(content="hey can you send me that report by tomorrow thx"),
]

ai_response = llm.invoke(messages)
print(ai_response.content)

Here's a corrected version of the text:

"Hey, can you send me that report by tomorrow, please?"

I made the following corrections:

* Added a comma after "tomorrow" to separate the main clause from the request.
* Changed "thx" to "please," which is a more polite and formal way to end the message.
* Added a question mark at the end to indicate that the text is a question.

However, considering the context, it seems like the message might be more suitable as a statement:

"Hey, I'd appreciate it if you could send me that report by tomorrow, please."

This revised message still conveys the request but in a more polite and formal tone.


In [55]:
# followup conversation
messages.append(ai_response)

In [56]:
#Ask a follow-up question
messages.append(HumanMessage(content="can you make the tone a bit informal"))

In [57]:
ai_response = llm.invoke(messages)
# print(ai_response)
print(ai_response.content)

Here's a revised version of the message with a slightly more informal tone:

"Hey, can you send me that report by tomorrow? That'd be a huge help, thanks."

I made the following changes to create a more informal tone:

* Removed the "please" and replaced it with "thanks," which is a more casual way to express gratitude.
* Added a phrase ("That'd be a huge help") to make the message more conversational and friendly.
* Kept the question mark at the end to maintain a polite and considerate tone.

Alternatively, if you want it to be even more casual, you could simplify the message to:

"Hey, can you send me that report by tomorrow? Thanks!"

This version still conveys the request and shows appreciation for the person's help, but in a more relaxed and friendly way.


#### Doing without LangChain

In [58]:
#Without LangChain - how would we initialize our LLM?
from openai import OpenAI

model_name = "llama-3.1-8b-instant"
llm_api = OpenAI(api_key=os.environ["GROQ_API_KEY"], base_url="https://api.groq.com/openai/v1")

In [59]:
# #Below is LangChain's messages format
# messages = [

#     SystemMessage(content="Detect and correct all spelling, grammatical, and punctuation errors in the provided text.\
#                   Ensure proper sentence structure, clarity, and readability.\
#                   Retain the core message of the original text while making the necessary corrections"),
#     HumanMessage(content="hey can you send me that report by tomorrow thx"),
# ]

#Below is messages in OpenAI format
messages_openai = [
    {'role':"system", 'content':"Detect and correct all spelling, grammatical, and punctuation errors in the provided text.\
     Ensure proper sentence structure, clarity, and readability.\
     Retain the core message of the original text while making the necessary corrections"},
     
     {'role':"user", 'content':"hey can you send me that report by tomorrow thx"}
]


In [60]:
ai_response_openai = llm_api.chat.completions.create(model= model_name,
                                messages=messages_openai)

In [61]:
ai_response_openai

ChatCompletion(id='chatcmpl-d10d8052-4d2e-41d5-803c-1907034d3f8f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='There is a bit of informal tone to the request. Here\'s a revised version with proper spelling, grammar, punctuation, and sentence structure:\n\n"Could you please send me the report by tomorrow? Thank you."', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1751520543, model='llama-3.1-8b-instant', object='chat.completion', service_tier=None, system_fingerprint='fp_0f5c9bc037', usage=CompletionUsage(completion_tokens=43, prompt_tokens=89, total_tokens=132, completion_tokens_details=None, prompt_tokens_details=None, queue_time=0.049183392000000006, prompt_time=0.005491698, completion_time=0.095450679, total_time=0.100942377), usage_breakdown=None, x_groq={'id': 'req_01jz7c50czfqkb18tpsv44y5s8'})

In [62]:
ai_response_openai_formatted = ai_response_openai.choices[0].message.content
print(ai_response_openai_formatted)

There is a bit of informal tone to the request. Here's a revised version with proper spelling, grammar, punctuation, and sentence structure:

"Could you please send me the report by tomorrow? Thank you."


In [63]:
messages_openai

[{'role': 'system',
  'content': 'Detect and correct all spelling, grammatical, and punctuation errors in the provided text.     Ensure proper sentence structure, clarity, and readability.     Retain the core message of the original text while making the necessary corrections'},
 {'role': 'user',
  'content': 'hey can you send me that report by tomorrow thx'}]

In [64]:
#Append the AI message
messages_openai.append(
    {'role': "assistant",
     'content': ai_response_openai_formatted}
)

In [65]:
messages_openai

[{'role': 'system',
  'content': 'Detect and correct all spelling, grammatical, and punctuation errors in the provided text.     Ensure proper sentence structure, clarity, and readability.     Retain the core message of the original text while making the necessary corrections'},
 {'role': 'user',
  'content': 'hey can you send me that report by tomorrow thx'},
 {'role': 'assistant',
  'content': 'There is a bit of informal tone to the request. Here\'s a revised version with proper spelling, grammar, punctuation, and sentence structure:\n\n"Could you please send me the report by tomorrow? Thank you."'}]

In [66]:
#Ask a follow-up question
#LangChain version below
# messages.append(HumanMessage(content="can you make the tone a bit informal"))

#OpenAI version below
messages_openai.append(
    {'role':"user",
     'content':"can you make the tone a bit informal"}
)

In [67]:
messages_openai

[{'role': 'system',
  'content': 'Detect and correct all spelling, grammatical, and punctuation errors in the provided text.     Ensure proper sentence structure, clarity, and readability.     Retain the core message of the original text while making the necessary corrections'},
 {'role': 'user',
  'content': 'hey can you send me that report by tomorrow thx'},
 {'role': 'assistant',
  'content': 'There is a bit of informal tone to the request. Here\'s a revised version with proper spelling, grammar, punctuation, and sentence structure:\n\n"Could you please send me the report by tomorrow? Thank you."'},
 {'role': 'user', 'content': 'can you make the tone a bit informal'}]

In [68]:
ai_response_openai = llm_api.chat.completions.create(
    model=model_name,
    messages=messages_openai
)

In [69]:
type(ai_response_openai)

openai.types.chat.chat_completion.ChatCompletion

In [70]:
print(ai_response_openai.choices[0].message.content)

Here's a revised version with a slightly more informal tone:

"Hey, would you be able to send me the report by tomorrow? Thanks!"


In [71]:
#Explain concept of context length here - Context length = input tokens + completion tokens 

### Output Parsers

In [72]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

parser = StrOutputParser()

*StrOutputParser* is a runnable object.

In [73]:
result = llm.invoke(messages)

parser.invoke(result)

'Here\'s the revised message:\n\n"Hey, can you get me that report by tomorrow, please?"\n\nI made the following changes to make the tone less formal:\n\n* Removed the phrase "I\'d appreciate it if," which is a bit more formal.\n* Changed the wording to "get me" instead of "send me," which is a more casual way to ask for something.\n\nHowever, if you want to make it even more informal, you could try:\n\n"Hey, do you think you could send the report by tomorrow?"\n\nThis version is a bit more relaxed and conversational.'

In [74]:
result

AIMessage(content='Here\'s the revised message:\n\n"Hey, can you get me that report by tomorrow, please?"\n\nI made the following changes to make the tone less formal:\n\n* Removed the phrase "I\'d appreciate it if," which is a bit more formal.\n* Changed the wording to "get me" instead of "send me," which is a more casual way to ask for something.\n\nHowever, if you want to make it even more informal, you could try:\n\n"Hey, do you think you could send the report by tomorrow?"\n\nThis version is a bit more relaxed and conversational.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 249, 'total_tokens': 367, 'completion_time': 0.196930076, 'prompt_time': 0.014841967, 'queue_time': 0.04843691900000001, 'total_time': 0.211772043}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8ab2e50475', 'finish_reason': 'stop', 'logprobs': None}, id='run--bbceeb0d-adcf-4eed-acf8-0aba034c234e-0', usage_metadata={'input_tokens': 249, 

In [75]:
messages = [
    SystemMessage(content="""You are an expert in writing analysis. You will receive a message from a user, and your job is to evaluate the text based on the following attributes:
1. clarity: Is the message clear, or unclear?
2. grammar_quality: Are there any grammatical issues? Possible values: correct, minor issues, major issues.
3. tone: Analyze whether the tone is neutral, formal, or informal.
4. suggestions: Offer brief improvement suggestions for clarity, grammar, or tone.

Return a structured JSON object with these four attributes."""),
    HumanMessage(content="Hey, could you please send me that report by tomorrow? Thank you.")
]
response = llm.invoke(messages)

In [76]:
response

AIMessage(content='Here\'s the analysis of the message:\n\n```json\n{\n  "clarity": "unclear",\n  "grammar_quality": "minor issues",\n  "tone": "informal",\n  "suggestions": [\n    "For clarity, please specify what report you are referring to.",\n    "Consider rephrasing to \'I would greatly appreciate it if you could send me the report by tomorrow\' for a clearer tone.",\n    "There is a misplaced period at the end of the message. For better grammar, consider ending the message with a period or a question mark."\n  ]\n}\n```\n\nThis analysis suggests that the message is unclear due to the lack of specifics about the report. The tone is informal, which may be suitable for a personal or casual conversation but may not be suitable for a formal request. There are minor grammatical issues, such as the misplaced period at the end of the message.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 184, 'prompt_tokens': 159, 'total_tokens': 343, 'completion_time': 

In [77]:
response.content

'Here\'s the analysis of the message:\n\n```json\n{\n  "clarity": "unclear",\n  "grammar_quality": "minor issues",\n  "tone": "informal",\n  "suggestions": [\n    "For clarity, please specify what report you are referring to.",\n    "Consider rephrasing to \'I would greatly appreciate it if you could send me the report by tomorrow\' for a clearer tone.",\n    "There is a misplaced period at the end of the message. For better grammar, consider ending the message with a period or a question mark."\n  ]\n}\n```\n\nThis analysis suggests that the message is unclear due to the lack of specifics about the report. The tone is informal, which may be suitable for a personal or casual conversation but may not be suitable for a formal request. There are minor grammatical issues, such as the misplaced period at the end of the message.'

In [92]:
json_response = JsonOutputParser().invoke(response.content)
json_response

{'clarity': 'unclear',
 'grammar_quality': 'minor issues',
 'tone': 'informal',
 'suggestions': ['For clarity, please specify what report you are referring to.',
  "Consider rephrasing to 'I would greatly appreciate it if you could send me the report by tomorrow' for a clearer tone.",
  'There is a misplaced period at the end of the message. For better grammar, consider ending the message with a period or a question mark.']}

In [93]:
var1 = 'json{"clarity": "unclear"}'
print(var1)

json{"clarity": "unclear"}


In [94]:
var1

'json{"clarity": "unclear"}'

In [96]:
# JsonOutputParser().invoke(var1)

In [97]:
type(json_response)

dict

In [86]:
type(var1)

str

In [79]:
print(response.content)

Here's the analysis of the message:

```json
{
  "clarity": "unclear",
  "grammar_quality": "minor issues",
  "tone": "informal",
  "suggestions": [
    "For clarity, please specify what report you are referring to.",
    "Consider rephrasing to 'I would greatly appreciate it if you could send me the report by tomorrow' for a clearer tone.",
    "There is a misplaced period at the end of the message. For better grammar, consider ending the message with a period or a question mark."
  ]
}
```

This analysis suggests that the message is unclear due to the lack of specifics about the report. The tone is informal, which may be suitable for a personal or casual conversation but may not be suitable for a formal request. There are minor grammatical issues, such as the misplaced period at the end of the message.


In [80]:
json_response['clarity']

'unclear'

### Chain (LCEL)

In [98]:
chain = llm | JsonOutputParser()

chain.invoke(messages)

{'clarity': 'clear',
 'grammar_quality': 'correct',
 'tone': 'informal',
 'suggestions': ['Consider including more details about the report or the expected format to ensure you receive the correct information.']}

### PromptTemplate

In [111]:
# my_str = "Hello world, this is the tone - {tone}"

In [112]:
# my_str.format(tone = "Happy")

In [113]:
# tone = "Happy"
# my_str = f"Hello world, this is the tone - {tone}"
# print(my_str)

In [101]:
my_str = """Detect and correct all spelling, grammatical, and punctuation errors in the provided text.
Ensure proper sentence structure, clarity, and readability.

Tone Adjustment: {tone}

Communication Style: {communication_style}

Retain the core message of the original text while making the necessary corrections and tone adjustments.
Suggest appropriate phrasing and formatting based on the tone and communication style."""

In [105]:
print(my_str.format(tone="Happy", communication_style = "Formal"))

Detect and correct all spelling, grammatical, and punctuation errors in the provided text.
Ensure proper sentence structure, clarity, and readability.

Tone Adjustment: Happy

Communication Style: Formal

Retain the core message of the original text while making the necessary corrections and tone adjustments.
Suggest appropriate phrasing and formatting based on the tone and communication style.


In [114]:
from langchain_core.prompts import ChatPromptTemplate
system_message_template = """Detect and correct all spelling, grammatical, and punctuation errors in the provided text.
Ensure proper sentence structure, clarity, and readability.

Tone Adjustment: {tone}

Communication Style: {communication_style}

Retain the core message of the original text while making the necessary corrections and tone adjustments.
Suggest appropriate phrasing and formatting based on the tone and communication style."""
template = ChatPromptTemplate([
    ("system", system_message_template),
    ("human", "{user_input}"),
])

In [115]:
template.input_variables

['communication_style', 'tone', 'user_input']

In [32]:
#How does this work without LCEL = LangChain Expression LangChain

In [116]:
tone = 'Rewrite the message in a professional, polite, and structured manner. \
Suitable for business emails, official reports, or any context requiring formality and respect.'

communication_style = 'Messages should be clear, structured, and formal or neutral depending on the context. \
Introductions, conclusions, and appropriate sign-offs should be added if missing.'

user_input = """Can u send me the data by eod pls?"""

In [117]:
#First step
formatted_template = template.invoke({"communication_style": communication_style,
    "tone":tone,
    "user_input":user_input
})
formatted_template

ChatPromptValue(messages=[SystemMessage(content='Detect and correct all spelling, grammatical, and punctuation errors in the provided text.\nEnsure proper sentence structure, clarity, and readability.\n\nTone Adjustment: Rewrite the message in a professional, polite, and structured manner. Suitable for business emails, official reports, or any context requiring formality and respect.\n\nCommunication Style: Messages should be clear, structured, and formal or neutral depending on the context. Introductions, conclusions, and appropriate sign-offs should be added if missing.\n\nRetain the core message of the original text while making the necessary corrections and tone adjustments.\nSuggest appropriate phrasing and formatting based on the tone and communication style.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can u send me the data by eod pls?', additional_kwargs={}, response_metadata={})])

In [118]:
type(formatted_template)

langchain_core.prompt_values.ChatPromptValue

In [119]:
#Second step
response = llm.invoke(formatted_template)

In [120]:
type(response)

langchain_core.messages.ai.AIMessage

In [123]:
response.content

"Here's a rewritten version of the message in a professional, polite, and structured manner:\n\nDear [Recipient],\n\nI would appreciate it if you could provide me with the requested data by the end of the day (EOD) today. If there are any issues or concerns that may affect the timely delivery of the data, please let me know as soon as possible so we can discuss alternative arrangements.\n\nThank you for your prompt attention to this matter.\n\nBest regards,\n[Your Name]\n\nAlternatively, if you prefer a more concise version:\n\nDear [Recipient],\n\nCould you please provide the requested data by the end of the day today? If you anticipate any delays, kindly inform me as soon as possible so we can discuss alternative arrangements.\n\nThank you for your cooperation.\n\nBest regards,\n[Your Name]\n\nNote that the original message is quite brief, so the rewritten versions have been kept concise while maintaining a professional tone."

In [121]:
#Third step
final_parsed_result = StrOutputParser().invoke(response)
final_parsed_result

"Here's a rewritten version of the message in a professional, polite, and structured manner:\n\nDear [Recipient],\n\nI would appreciate it if you could provide me with the requested data by the end of the day (EOD) today. If there are any issues or concerns that may affect the timely delivery of the data, please let me know as soon as possible so we can discuss alternative arrangements.\n\nThank you for your prompt attention to this matter.\n\nBest regards,\n[Your Name]\n\nAlternatively, if you prefer a more concise version:\n\nDear [Recipient],\n\nCould you please provide the requested data by the end of the day today? If you anticipate any delays, kindly inform me as soon as possible so we can discuss alternative arrangements.\n\nThank you for your cooperation.\n\nBest regards,\n[Your Name]\n\nNote that the original message is quite brief, so the rewritten versions have been kept concise while maintaining a professional tone."

In [131]:
# #With LCEL
# proof_read_chain = template | llm ##RunnableSequence
# final_response = proof_read_chain.invoke(
#     {"communication_style": communication_style,
#     "tone":tone,
#     "user_input":user_input
# })
# final_response
# type(final_response)

In [134]:
#With LCEL
proof_read_chain = template | llm | StrOutputParser()
proof_read_chain

ChatPromptTemplate(input_variables=['communication_style', 'tone', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['communication_style', 'tone'], input_types={}, partial_variables={}, template='Detect and correct all spelling, grammatical, and punctuation errors in the provided text.\nEnsure proper sentence structure, clarity, and readability.\n\nTone Adjustment: {tone}\n\nCommunication Style: {communication_style}\n\nRetain the core message of the original text while making the necessary corrections and tone adjustments.\nSuggest appropriate phrasing and formatting based on the tone and communication style.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11670de50>, async_client=<groq.

In [135]:
type(proof_read_chain)

langchain_core.runnables.base.RunnableSequence

In [133]:
final_response = proof_read_chain.invoke(
    {"communication_style": communication_style,
    "tone":tone,
    "user_input":user_input
})
final_response

"Here's a rewritten version of the message in a professional and polite tone:\n\nDear [Recipient],\n\nI kindly request that you provide me with the necessary data by the end of the day (EOD) today. Please let me know if this deadline is feasible and if there are any issues that may delay the data delivery.\n\nI appreciate your prompt attention to this matter and look forward to receiving the data by the designated time.\n\nThank you for your cooperation and assistance.\n\nBest regards,\n[Your Name]"

In [136]:
# proof_read_chain.input_schema.model_json_schema()

In [39]:
tone_map = {
    "Formal": "Rewrite the message in a professional, polite, and structured manner. Suitable for business emails, official reports, or any context requiring formality and respect.",
    "Informal": "Rewrite in a casual, friendly, and conversational style. Appropriate for personal communications, friendly chats, or informal emails.",
    "Neutral": "Rewrite in a balanced tone that is neither overly formal nor too casual. Suitable for most general communications where a middle-ground tone is required."
}
communication_style_map = {
    "Email": "Messages should be clear, structured, and formal or neutral depending on the context. Introductions, conclusions, and appropriate sign-offs should be added if missing.",
    "General": "This covers most forms of communication and will aim for clarity and coherence. The tone can vary as per the user's choice.",
    "Instant Messaging": "Focus on brevity, clarity, and informality, using conversational phrasing suitable for quick back-and-forth exchanges.",
    "Business Instant Messaging": "Maintain a professional but conversational tone. Messages should be concise and efficient, avoiding unnecessary formalities but keeping the language respectful."
}

In [40]:
tone = "Formal" # Formal, Informal, Neutral
communication_style = "Email" # Email, General, Instant Messaging, Business Instant Messaging
user_input = """Can u send me the data by eod pls?"""

chain_output = proof_read_chain.invoke(dict(
    tone=tone_map[tone],
    communication_style = communication_style_map[communication_style],
    user_input = user_input
))
print(chain_output)

Here is a rewritten version of your message in a professional, polite, and structured tone:

"Dear [Recipient],

I would greatly appreciate it if you could provide me with the requested data by the end of the day (EOD) today. Could you please ensure that the data is sent to me in a timely manner so that I can review and utilize it accordingly?

Thank you for your prompt attention to this matter.

Best regards,
[Your Name]"

I made the following changes to the original message:

* Added a formal greeting and sign-off
* Changed the tone to be polite and professional
* Structured the message to be clear and concise
* Retained the core message of the original text while making necessary corrections
